# Laboratorium nr 1

## Konfiguracja


Poniższe narzędzie zostało zmodyfikowane, tak aby można było z niego korzystać w nowszych IDE, które nie wspierają `%matplotlib notebook`.
Aby korzystać z `%matplotlib widget` należy doinstalować pakiet `ipympl` np. za pomocą polecenia `pip install ipympl`. Jeżeli po zainstalowaniu będą występować błędy, należy korzystać z backendu `notebook`.

Wykorzystywane biblioteki:
- matplotlib
- numpy
- ipympl
- time - do mierzenia czasu liczenia wyznaczników

Nowe funkcje:
- Dodanie do konstruktora klasy Scene argumentu `title`, który pozwala na ustawienie tytułu danej sceny
- Dodanie do konstruktora klasy Plot argumentu `title`, który pozwala na ustawienie tytułu wykresu / okna widżetu

W celu powtarzalności wyników (dla zgodności z sprawozdaniem) ustawiono ziarno (seed) dla funkcji typu random, osobno dla każdego zbioru danych.

In [ ]:
# Wybór backendu matplotlib
# %matplotlib notebook
%matplotlib widget

# Narzędzie jest oparte o kilka zewnętrznych bibliotek, które potrzebujemy najpierw zaimportować.
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import json as js
import time
from statistics import mean

# Parametr określający jak blisko (w odsetku całego widocznego zakresu) punktu początkowego 
# wielokąta musimy kliknąć, aby go zamknąć.
TOLERANCE = 0.15

def dist(point1, point2):
    return np.sqrt(np.power(point1[0] - point2[0], 2) + np.power(point1[1] - point2[1], 2))

# Klasa ta trzyma obecny stan wykresu oraz posiada metody, które mają zostać wykonane
# po naciśnięciu przycisków.
class _Button_callback(object):
    def __init__(self, scenes):
        self.i = 0
        self.scenes = scenes
        self.adding_points = False
        self.added_points = []
        self.adding_lines = False
        self.added_lines = []
        self.adding_rects = False
        self.added_rects = []
        # Dodano: zapisz tytuł sceny
        self.scene_title = self.scenes[0].title if len(self.scenes) and self.scenes[0].title else ""

    def set_axes(self, ax):
        self.ax = ax
        
    # Metoda ta obsługuje logikę przejścia do następnej sceny.
    def next(self, event):
        self.i = (self.i + 1) % len(self.scenes)
        # Dodano: zmień tytuł sceny
        self.scene_title = self.scenes[self.i].title
        self.draw(autoscaling = True)

    # Metoda ta obsługuje logikę powrotu do poprzedniej sceny.
    def prev(self, event):
        self.i = (self.i - 1) % len(self.scenes)
        # Dodano: zmień tytuł sceny
        self.scene_title = self.scenes[self.i].title
        self.draw(autoscaling = True)
        
    # Metoda ta aktywuje funkcję rysowania punktów wyłączając równocześnie rysowanie 
    # odcinków i wielokątów.
    def add_point(self, event):
        self.adding_points = not self.adding_points
        self.new_line_point = None
        if self.adding_points:
            self.adding_lines = False
            self.adding_rects = False
            self.added_points.append(PointsCollection([]))
            
    # Metoda ta aktywuje funkcję rysowania odcinków wyłączając równocześnie
    # rysowanie punktów i wielokątów.     
    def add_line(self, event):   
        self.adding_lines = not self.adding_lines
        self.new_line_point = None
        if self.adding_lines:
            self.adding_points = False
            self.adding_rects = False
            self.added_lines.append(LinesCollection([]))

    # Metoda ta aktywuje funkcję rysowania wielokątów wyłączając równocześnie
    # rysowanie punktów i odcinków.
    def add_rect(self, event):
        self.adding_rects = not self.adding_rects
        self.new_line_point = None
        if self.adding_rects:
            self.adding_points = False
            self.adding_lines = False
            self.new_rect()
    
    def new_rect(self):
        self.added_rects.append(LinesCollection([]))
        self.rect_points = []
        
    # Metoda odpowiedzialna za właściwą logikę rysowania nowych elementów. W
    # zależności od włączonego trybu dodaje nowe punkty, początek, koniec odcinka
    # lub poszczególne wierzchołki wielokąta. Istnieje ciekawa logika sprawdzania
    # czy dany punkt jest domykający dla danego wielokąta. Polega ona na tym, że
    # sprawdzamy czy odległość nowego punktu od początkowego jest większa od
    # średniej długości zakresu pomnożonej razy parametr TOLERANCE.   
    def on_click(self, event):
        if event.inaxes != self.ax:
            return
        new_point = (event.xdata, event.ydata)
        if self.adding_points:
            self.added_points[-1].add_points([new_point])
            self.draw(autoscaling = False)
        elif self.adding_lines:
            if self.new_line_point is not None:
                self.added_lines[-1].add([self.new_line_point, new_point])
                self.new_line_point = None
                self.draw(autoscaling = False)
            else:
                self.new_line_point = new_point
        elif self.adding_rects:
            if len(self.rect_points) == 0:
                self.rect_points.append(new_point)
            elif len(self.rect_points) == 1:
                self.added_rects[-1].add([self.rect_points[-1], new_point])
                self.rect_points.append(new_point)
                self.draw(autoscaling = False)
            elif len(self.rect_points) > 1:
                if dist(self.rect_points[0], new_point) < (np.mean([self.ax.get_xlim(), self.ax.get_ylim()])*TOLERANCE):
                    self.added_rects[-1].add([self.rect_points[-1], self.rect_points[0]])
                    self.new_rect()
                else:    
                    self.added_rects[-1].add([self.rect_points[-1], new_point])
                    self.rect_points.append(new_point)
                self.draw(autoscaling = False)
    
    # Metoda odpowiedzialna za narysowanie całego wykresu. Warto zauważyć,
    # że zaczyna się ona od wyczyszczenia jego wcześniejszego stanu. Istnieje w
    # niej nietrywialna logika zarządzania zakresem wykresu, tak żeby, w zależności
    # od ustawionego parametru autoscaling, uniknąć sytuacji, kiedy dodawanie
    # nowych punktów przy brzegu obecnie widzianego zakresu powoduje niekorzystne
    # przeskalowanie.
    def draw(self, autoscaling = True):
        if not autoscaling:
            xlim = self.ax.get_xlim()
            ylim = self.ax.get_ylim()
        self.ax.clear()
        for collection in (self.scenes[self.i].points + self.added_points):
            if len(collection.points) > 0:
                self.ax.scatter(*zip(*(np.array(collection.points))), **collection.kwargs)
        for collection in (self.scenes[self.i].lines + self.added_lines + self.added_rects):
            self.ax.add_collection(collection.get_collection())
        self.ax.autoscale(autoscaling)
        if not autoscaling:
            self.ax.set_xlim(xlim)
            self.ax.set_ylim(ylim)
            
        # Dodano: zapisz tytuł sceny
        fig = plt.gcf() # Aktywne okno
        fig.suptitle(self.scene_title)
        
        plt.draw()


### Interfejsy

[Dostępne kolory](https://matplotlib.org/3.1.1/gallery/color/named_colors.html)

[Dostępne znaczniki punktów](https://matplotlib.org/3.1.1/api/markers_api.html#module-matplotlib.markers)

In [ ]:
# Klasa Scene odpowiada za przechowywanie elementów, które mają być
# wyświetlane równocześnie. Konkretnie jest to lista PointsCollection i
# LinesCollection.
class Scene:
    def __init__(self, points=[], lines=[], title=""):
        self.points=points
        self.lines=lines
        # Dodano: zapisz tytuł sceny
        self.title = '\n'+title if title != "" else ""

# Klasa PointsCollection gromadzi w sobie punkty jednego typu, a więc takie,
# które zostaną narysowane w takim samym kolorze i stylu. W konstruktorze
# przyjmuje listę punktów rozumianych jako pary współrzędnych (x, y). Parametr
# kwargs jest przekazywany do wywołania funkcji z biblioteki MatPlotLib przez
# co użytkownik może podawać wszystkie parametry tam zaproponowane.        
class PointsCollection:
    def __init__(self, points, **kwargs):
        self.points = points
        self.kwargs = kwargs
    
    def add_points(self, points):
        self.points = self.points + points

# Klasa LinesCollection podobnie jak jej punktowy odpowiednik gromadzi
# odcinki tego samego typu. Tworząc ją należy podać listę linii, gdzie każda
# z nich jest dwuelementową listą punktów – par (x, y). Parametr kwargs jest
# przekazywany do wywołania funkcji z biblioteki MatPlotLib przez co użytkownik
# może podawać wszystkie parametry tam zaproponowane.
class LinesCollection:
    def __init__(self, lines, **kwargs):
        self.lines = lines
        self.kwargs = kwargs
        
    def add(self, line):
        self.lines.append(line)
        
    def get_collection(self):
        return mcoll.LineCollection(self.lines, **self.kwargs)

# Klasa Plot jest najważniejszą klasą w całym programie, ponieważ agreguje
# wszystkie przygotowane sceny, odpowiada za stworzenie wykresu i przechowuje
# referencje na przyciski, dzięki czemu nie będą one skasowane podczas tzw.
# garbage collectingu.
class Plot:
    def __init__(self, scenes = [Scene()], points = [], lines = [], json = None, title=""):
        # ADDED: plot title
        self.plot_title = title
        if json is None:
            self.scenes = scenes
            if points or lines:
                self.scenes[0].points = points
                self.scenes[0].lines = lines
        else:
            self.scenes = [Scene([PointsCollection(pointsCol) for pointsCol in scene["points"]], 
                                 [LinesCollection(linesCol) for linesCol in scene["lines"]]) 
                           for scene in js.loads(json)]
    
    # Ta metoda ma szczególne znaczenie, ponieważ konfiguruje przyciski i
    # wykonuje tym samym dość skomplikowaną logikę. Zauważmy, że konfigurując każdy
    # przycisk podajemy referencję na metodę obiektu _Button_callback, która
    # zostanie wykonana w momencie naciśnięcia.
    def __configure_buttons(self):
        plt.subplots_adjust(bottom=0.2)
        ax_prev = plt.axes([0.6, 0.05, 0.15, 0.075])
        ax_next = plt.axes([0.76, 0.05, 0.15, 0.075])
        ax_add_point = plt.axes([0.44, 0.05, 0.15, 0.075])
        ax_add_line = plt.axes([0.28, 0.05, 0.15, 0.075])
        ax_add_rect = plt.axes([0.12, 0.05, 0.15, 0.075])
        b_next = Button(ax_next, 'Następny')
        b_next.on_clicked(self.callback.next)
        b_prev = Button(ax_prev, 'Poprzedni')
        b_prev.on_clicked(self.callback.prev)
        b_add_point = Button(ax_add_point, 'Dodaj punkt')
        b_add_point.on_clicked(self.callback.add_point)
        b_add_line = Button(ax_add_line, 'Dodaj linię')
        b_add_line.on_clicked(self.callback.add_line)
        b_add_rect = Button(ax_add_rect, 'Dodaj figurę')
        b_add_rect.on_clicked(self.callback.add_rect)
        return [b_prev, b_next, b_add_point, b_add_line, b_add_rect]
    
    def add_scene(self, scene):
        self.scenes.append(scene)
    
    def add_scenes(self, scenes):
        self.scenes = self.scenes + scenes

    # Metoda toJson() odpowiada za zapisanie stanu obiektu do ciągu znaków w
    # formacie JSON.
    def toJson(self):
        return js.dumps([{"points": [np.array(pointCol.points).tolist() for pointCol in scene.points], 
                          "lines":[linesCol.lines for linesCol in scene.lines]} 
                         for scene in self.scenes])    
    
    # Metoda ta zwraca punkty dodane w trakcie rysowania.
    def get_added_points(self):
        if self.callback:
            return self.callback.added_points
        else:
            return None
    
    # Metoda ta zwraca odcinki dodane w trakcie rysowania.
    def get_added_lines(self):
        if self.callback:
            return self.callback.added_lines
        else:
            return None
        
    # Metoda ta zwraca wielokąty dodane w trakcie rysowania.
    def get_added_figure(self):
        if self.callback:
            return self.callback.added_rects
        else:
            return None
    
    # Metoda ta zwraca punkty, odcinki i wielokąty dodane w trakcie rysowania
    # jako scenę.
    def get_added_elements(self):
        if self.callback:
            return Scene(self.callback.added_points, self.callback.added_lines+self.callback.added_rects)
        else:
            return None
    
    # Główna metoda inicjalizująca wyświetlanie wykresu.
    def draw(self):
        plt.close()
        # Dodano: tytuł wykresu
        fig = plt.figure(num=self.plot_title)
        self.callback = _Button_callback(self.scenes)
        self.widgets = self.__configure_buttons()
        ax = plt.axes(autoscale_on = False)
        self.callback.set_axes(ax)
        fig.canvas.mpl_connect('button_press_event', self.callback.on_click)
        plt.show()
        self.callback.draw()
        

### Przykłady użycia

##### Proste rysowanie

Rysowanie prostego rysunku złożonego ze statycznych punktów i odcinków jest stosunkowo proste. Należy utworzyć zmienną z obiektem `Plot`, który przyjmuje w konstruktorze listę obiektów `PointsCollection` jako parametr `points` oraz listę `LinesCollection` jako parametr `lines`. Następnie należy wywołać jego metodę `draw()`.

```python
plot = Plot(points=[PointsCollection([(1, 2), (3, 1.5), (2, -1)]),
               PointsCollection([(5, -2), (2, 2), (-2, -1)], color='green', marker = "^")],
            lines=[LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]])])
plot.draw()     
```

##### Rysunek dynamiczny

To narzędzie pozwala na wizualizację "rysunków dynamicznych", a więc takich, które składają się z wielu różnych obrazów. Może to być przydatne np. podczas wizualizacji kolejnych kroków danego algorytmu. Poszczególne klatki określane są jako obiekty `Scene`, które przyjmują listę `PointsCollection` oraz listę `LinesCollection` w swoim konstruktorze. Obiekt `Plot` przyjmuje w parametrze konstruktora `scenes` właśnie listę takich scen.

Na rysunku przyciski `Następny` i `Poprzedni` służą do nawigacji pomiędzy nimi.

```python
scenes=[Scene([PointsCollection([(1, 2), (3, 1.5), (2, -1)]), 
               PointsCollection([(5, -2), (2, 2), (-2, -1)], color='green', marker = "^")], 
              [LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]])], "Scene 1"), 
        Scene([PointsCollection([(1, 2), (3, 1.5), (2, -1)], color='red'), 
               PointsCollection([(5, -2), (2, 2), (-2, 1)], color='black')], 
              [LinesCollection([[(-1,2),(-2,3)], [(0,-1),(-1,0)]])],"Scene 2")]

plot=Plot(scenes = scenes)
plot.draw()
```

##### Zapis i odczyt z pliku

Klasa Plot posiada metodę `toJson()`, która zwraca string zawierający listę scen w formacie JSON. Taki string można normalnie zapisać do pliku stosując normalne sposoby dostępne w Pythonie. Wczytanie listy scen z pliku dokonuje się poprzez podanie parametru `json` w kostruktorze `Plot`.

```python
scenes=[Scene([PointsCollection([(-2, -1)], color='green', marker = "^")], 
              [LinesCollection([[(1,2),(2,3)], [(0,1),(1,0)]])]), 
        Scene([PointsCollection([(1, 2), (3, 1.5)], color='red'), 
               PointsCollection([(5, -2)], color='black')])]
plot = Plot(scenes)

with open('somefile.json', 'w') as file:
    file.write(plot.toJson())
    
#somefile.json: [{"points": [[[-2, -1]]], "lines": [[[[1, 2], [2, 3]], [[0, 1], [1, 0]]]]}, {"points": [[[1.0, 2.0], [3.0, 1.5]], [[5, -2]]], "lines": []}]
    
with open('somefile.json', 'r') as file:
    json = file.read()
    
plot2 = Plot(json=json)
plot2.draw()
```

##### Operowanie na dodanych punktach

Możemy bardzo łatwo rysować nowe punkty i linie. Aby je później wykorzystać sugerowanym sposobem jest utworzenie nowej komórki, w której utworzymy nowy `Plot` i skorzystamy z metod `get_added_points()` i `get_added_points()` lub `get_added_elements()` (zwraca scenę) z pierwszego `Plot`u.

```python 
#Dodaj tu punkty, figury i/lub odcinki!
plot1 = Plot(points=[PointsCollection([(-2,-1),(5,3)], color='red')])
plot1.draw()

plot2 = Plot([plot1.get_added_elements()])
plot2.draw()
```

## Zbiory punktów z domyślną precyzją


Zbiór A -  $10^5$ losowych punktów o współrzędnych z przedziału $[-1000, 1000]$

In [ ]:
# Set seed for reproducibility
np.random.seed(2317)

points_list_a = np.random.uniform(-1000,1000, size=[10 ** 5, 2])
points_list_a = PointsCollection(points_list_a)
plot = Plot(points=[points_list_a], title="Zestaw danych A")
plot.draw()

Zbiór B - $10^5$ losowych punktów o współrzędnych z przedziału $[-10^{14}, 10^{14}]$

In [ ]:
# Set seed for reproducibility
np.random.seed(2173)

points_list_b = np.random.uniform(-(10 ** 14),10 ** 14, size=[10 ** 5, 2])
points_list_b = PointsCollection(points_list_b)
plot = Plot(points=[points_list_b], title="Zestaw danych B")
plot.draw()

Zbiór C - $1000$ losowych punktów leżących na okręgu o środku $(0,0)$ i promieniu $R=100$

In [ ]:
# Set seed for reproducibility
np.random.seed(3217)

# Generate points on a circle using polar coordinates: (Rsinx, Rcosx), x in [0, 2pi]
points_list = []
R = 100
random_list = np.random.uniform(0,2*np.pi, size=1000)

for i in range(1000):
    x = random_list[i]
    points_list.append((R*np.sin(x), R*np.cos(x)))

points_list_c = PointsCollection(points_list)
plot = Plot(points=[points_list_c], title="Zestaw danych C")
plot.draw()

Zbiór D - $1000$ losowych punktów o współrzędnych z przedziału $[-1000, 1000]$ leżących na prostej
wyznaczonej przez wektor (a, b).  
Przyjmij $a = [-1.0, 0.0]$, $b = [1.0, 0.1]$

In [ ]:
# Set seed for reproducibility
np.random.seed(7312)

A = (-1.0,0.0)
B = (1.0,0.1)

# y = a*x + b
a = (A[1] - B[1])/(A[0] - B[0])
b = A[1] - a*A[0]


points_list = []
random_points = np.random.uniform(-1000,1000, size=1000)

for i in range(1000):    
    x = random_points[i]
    y = a*x + b
    points_list.append((x,y))

points_list_d = PointsCollection(points_list)
plot = Plot(points=[points_list_d], title="Zestaw danych D")
plot.draw()


### Klasyfikacja punktów

#### Implementacje wyznaczników

Do klasyfikacji punktów wykorzystamy własne implementacje wyznaczników, które wykorzystują metodę Sarrusa oraz wyznaczniki dostępne w bibliotece `numpy`.

In [ ]:
def det3x3(a,b,c,numpy=True):
    matrix = [[],[],[]]
    matrix[0] = [a[0], a[1], 1]
    matrix[1] = [b[0], b[1], 1]
    matrix[2] = [c[0], c[1], 1]

    if numpy:
        matrix = np.array(matrix)
        return np.linalg.det(matrix)

    f1 = matrix[0][0] * matrix[1][1] * matrix[2][2]
    f2 = matrix[0][1] * matrix[1][2] * matrix[2][0]
    f3 = matrix[1][0] * matrix[2][1] * matrix[0][2]
    f4 = matrix[0][2] * matrix[1][1] * matrix[2][0]
    f5 = matrix[1][2] * matrix[2][1] * matrix[0][0]
    f6 = matrix[1][0] * matrix[0][1] * matrix[2][2]
    return f1+f2+f3-f4-f5-f6

def det2x2(a,b,c,numpy=True):
    matrix = [[],[]]
    matrix[0] = [a[0]-c[0], a[1]-c[1]]
    matrix[1] = [b[0]-c[0], b[1]-c[1]]

    if numpy:
        matrix = np.array(matrix)
        return np.linalg.det(matrix)

    f1 = matrix[0][0] * matrix[1][1]
    f2 = matrix[0][1] * matrix[1][0]
    return f1-f2

#### Funkcje pomocnicze
Za klasyfikację punktów odpowiada funkcja `classify_points`, która pozwala klasyfikować punkty z dokładnością do podanej tolerancji. Funkcja zwraca sklasyfikowane punkty, które są rozszerzone o informacje po której stronie odcinka są oraz listę z punktami sklasyfikowanymi po lewej, prawej stronie odcinka lub na odcinku.
Legenda kolorów:
- czerwony - punkt jest współliniowy
- niebieski - punkt jest po lewej stronie
- zielony - punkt jest po prawej stronie

Funkcja `print_classified_points` odpowiada za wygenerowanie scen do wykresu oraz do stworzenia listy z danymi potrzebnej do utworzenia tabeli porównującej dane wyznaczniki.

Aby pokazać tabelę porównującą dane wyznaczniki wraz z odpowiednimi tolerancjami zera należy wywołać funkcję `print_classified_points_table`, która przyjmuje dane z funkcji `print_classified_points`

Aby zmienić wykres danej klasyfikacji punktów należy kliknąć odpowiednio `Następny` lub `Poprzedni`. Po naciśnięciu tytuł wykresu powinien zostać zmieniony.

In [ ]:
def classify_points(points,det_function, use_numpy=True, epsilon=0):
    # Segment points
    a = [-1.0,0.0]
    b = [1.0, 0.1]  
    
    collinear = []
    left = []
    right = []
    clasified_points = []

    measured_time = []

    for c in points.points:
        # Time measurement
        t = time.process_time()
        det_value = det_function(a,b,c,use_numpy)
        measured_time.append(time.process_time() - t)
        if -epsilon <= det_value <= epsilon:
            collinear.append(c)
            clasified_points.append((0,c[0],c[1]))
        elif det_value < -epsilon:
            right.append(c)
            clasified_points.append((-1,c[0],c[1]))
        else:
            left.append(c)
            clasified_points.append((1,c[0],c[1]))

    left = PointsCollection(left, color='blue')
    right = PointsCollection(right, color='green')
    collinear = PointsCollection(collinear, color='red')
    
    return clasified_points, [left, right, collinear], mean(measured_time)

def print_classified_points(points_list):
    precisions = [1e-18, 1e-14, 1e-10, 1e-06]
    scenes = []
    # Determinant order: [własny 2x2, numpy 2x2, własny 3x3, numpy 3x3]
    # Precision order:  [1e-18, 1e-14, 1e-10, 1e-06]
    classified_points = []
    
    for prec in precisions:
        classified_points_group = []

        classified_points_2m, points_2m, avg_time_2m = classify_points(points_list, det2x2, use_numpy=False, epsilon=prec)
        classified_points_group.append((classified_points_2m, len(points_2m[0].points), len(points_2m[1].points), 
                                        len(points_2m[2].points), prec,"Wyznacznik własny 2x2",avg_time_2m))

        classified_points_2n, points_2n,avg_time_2n = classify_points(points_list, det2x2, use_numpy=True, epsilon=prec)
        classified_points_group.append((classified_points_2n, len(points_2n[0].points), len(points_2n[1].points), 
                                        len(points_2n[2].points), prec,"Wyznacznik numpy 2x2",avg_time_2n))

        classified_points_3m, points_3m,avg_time_3m = classify_points(points_list, det3x3, use_numpy=False, epsilon=prec)
        classified_points_group.append((classified_points_3m, len(points_3m[0].points), len(points_3m[1].points), 
                                        len(points_3m[2].points), prec,"Wyznacznik własny 3x3",avg_time_3m))

        classified_points_3n, points_3n,avg_time_3n = classify_points(points_list, det3x3, use_numpy=True, epsilon=prec)
        classified_points_group.append((classified_points_3n, len(points_3n[0].points), len(points_3n[1].points), 
                                        len(points_3n[2].points), prec,"Wyznacznik numpy 3x3",avg_time_3n))

        scenes.append(Scene(points=points_2m, title=f"Wyznacznik własny 2x2 tolerancja {prec}"))
        scenes.append(Scene(points=points_2n, title=f"Wyznacznik numpy 2x2 tolerancja {prec}"))
        scenes.append(Scene(points=points_3m, title=f"Wyznacznik własny 3x3 tolerancja {prec}"))
        scenes.append(Scene(points=points_3n, title=f"Wyznacznik numpy 3x3 tolerancja {prec}"))
        
        classified_points.append(classified_points_group)
        
    return scenes, classified_points

def print_classified_points_table(classified_points, table_title = ""):
    print(table_title + '\n')
    for prec in range(len(classified_points)):
        title = f"Epsilon {classified_points[prec][0][-3]}"
        title = title.center(120,'-')
        print(title)
        for i in range(len(classified_points[0])):
            col1 = f"{classified_points[prec][i][-2]}"
            col1 = col1.center(25)
            col2 = f"Left: {classified_points[prec][i][1]}"
            col2 = col2.center(18)
            col3 = f"Right: {classified_points[prec][i][2]}"
            col3 = col3.center(18)
            col4 = f"Collinear: {classified_points[prec][i][3]}"
            col4 = col4.center(18)
            col5 = f"Average CPU time: {classified_points[prec][i][-1]:.5e}"
            col5 = col5.center(36)
            print(f"|{col1}|{col2}|{col3}|{col4}|{col5}|")
            print(120 * '-')   

#### Zestaw danych A

In [ ]:
scenes, classified_points_a = print_classified_points(points_list_a)
plot = Plot(scenes, title=f"Klasyfikacja punktów zestaw danych A")
plot.draw()

print_classified_points_table(classified_points_a, "Tabela klasyfikacji punktów względem tolerancji zera:")

#### Zestaw danych B

In [ ]:
scenes, classified_points_b = print_classified_points(points_list_b)
plot = Plot(scenes, title=f"Klasyfikacja punktów zestaw danych B")
plot.draw()

print_classified_points_table(classified_points_b, "Tabela klasyfikacji punktów względem tolerancji zera:")

#### Zestaw danych C

In [ ]:
scenes, classified_points_c = print_classified_points(points_list_c)
plot = Plot(scenes, title=f"Klasyfikacja punktów zestaw danych C")
plot.draw()

print_classified_points_table(classified_points_c, "Tabela klasyfikacji punktów względem tolerancji zera:")

#### Zestaw danych D

In [ ]:
scenes, classified_points_d = print_classified_points(points_list_d)
plot = Plot(scenes, title=f"Klasyfikacja punktów zestaw danych D")
plot.draw()

print_classified_points_table(classified_points_d, "Tabela klasyfikacji punktów względem tolerancji zera:")

### Porównanie klasyfikacji

Funkcja `compare_points`, pozwala na porównanie punktów, które zwróciła funkcja `classify_points`, dla danego zestawu danych. Funkcja zwraca gotową listę punktów typu `PointsCollection`.

Aby wygenerować sceny oraz dane do tabeli porównującej różnice w klasyfikacji punktów należy użyć funkcji `print_compare_points`.

Tabela wypisująca ile punktów zostało inaczej zakwalifikowanych jest zaimplementowana w funkcji `print_compared_points_table`

Aby zmienić wykres danego porównania klasyfikacji punktów należy kliknąć odpowiednio `Następny` lub `Poprzedni`. Po naciśnięciu tytuł wykresu powinien zostać zmieniony.

In [ ]:
def compare_points(p1,p2):
    different_points = []
    for i in range(len(p1)):
        t1,x1,y1 = p1[i]
        t2,x2,y2 = p2[i]

        if t1==t2:
            continue

        different_points.append((x1,y1))
    return  [PointsCollection(different_points,color='red')]

def print_compare_points(classified_points):
    scenes = []
    precisions = [1e-18, 1e-14, 1e-10 ,1e-06]
    # Determinant order:      [własny 2x2 i numpy 2x2, 
    #                          własny 3x3 i numpy 3x3, 
    #                          własny 2x2 i numpy 3x3, 
    #                          własny 3x3 i numpy 2x2]
    # Precision order:  [1e-18, 1e-14, 1e-10, 1e-06]
    compared_points_array = []
    for i in range(len(classified_points)): 
        prec = precisions[i]
        compared_points_group = []

        compared_points = compare_points(classified_points[i][0][0],classified_points[i][1][0])
        scenes.append(Scene(points=compared_points, title=f"Wyznacznik własny 2x2 i numpy 2x2 - {prec}"))
        compared_points_group.append((len(compared_points[0].points),"Wyznacznik własny 2x2 i numpy 2x2",prec))
        
        compared_points = compare_points(classified_points[i][2][0],classified_points[i][3][0])
        scenes.append(Scene(points=compared_points, title=f"Wyznacznik własny 3x3 i numpy 3x3 - {prec}"))
        compared_points_group.append((len(compared_points[0].points),"Wyznacznik własny 3x3 i numpy 3x3",prec))

        compared_points = compare_points(classified_points[i][0][0],classified_points[i][3][0])
        scenes.append(Scene(points=compared_points, title=f"Wyznacznik własny 2x2 i numpy 3x3 - {prec}"))
        compared_points_group.append((len(compared_points[0].points),"Wyznacznik własny 2x2 i numpy 3x3",prec))

        compared_points = compare_points(classified_points[i][2][0],classified_points[i][1][0])
        scenes.append(Scene(points=compared_points, title=f"Wyznacznik własny 3x3 i numpy 2x2 - {prec}"))
        compared_points_group.append((len(compared_points[0].points),"Wyznacznik własny 3x3 i numpy 2x2",prec))

        compared_points = compare_points(classified_points[i][0][0],classified_points[i][2][0])
        scenes.append(Scene(points=compared_points, title=f"Wyznacznik własny 2x2 i własny 3x3 - {prec}"))
        compared_points_group.append((len(compared_points[0].points),"Wyznacznik własny 2x2 i własny 3x3",prec))

        compared_points = compare_points(classified_points[i][1][0],classified_points[i][3][0])
        scenes.append(Scene(points=compared_points, title=f"Wyznacznik numpy 2x2 i numpy 3x3 - {prec}"))
        compared_points_group.append((len(compared_points[0].points),"Wyznacznik numpy 2x2 i numpy 3x3",prec))

        compared_points_array.append(compared_points_group)

    return scenes, compared_points_array


def print_compared_points_table(classified_points, table_title = ""):
    print(table_title + '\n')
    for prec in range(len(classified_points)):
        title = f"Epsilon {classified_points[prec][0][-1]}"
        title = title.center(63,'-')
        print(title)
        for i in range(len(classified_points[0])):
            col1 = f"{classified_points[prec][i][-2]}"
            col1 = col1.center(40)
            col2 = f"Difference: {classified_points[prec][i][0]}"
            col2 = col2.center(20)
            print(f"|{col1}|{col2}|")
            print(63 * '-')   

#### Zestaw danych A

In [ ]:
scenes, compared_points_a = print_compare_points(classified_points_a)
plot = Plot(scenes, title="Porównanie wyznaczników zestaw danych A")
plot.draw()

print_compared_points_table(compared_points_a, "Liczba punktów inaczej sklasyfikowanych przez wyznaczniki:")

#### Zestaw danych B

In [ ]:
scenes, compared_points_b = print_compare_points(classified_points_b)
plot = Plot(scenes, title="Porównanie wyznaczników zestaw danych B")
plot.draw()

print_compared_points_table(compared_points_b, "Liczba punktów inaczej sklasyfikowanych przez wyznaczniki:")

#### Zestaw danych C

In [ ]:
scenes, compared_points_c = print_compare_points(classified_points_c)
plot = Plot(scenes, title="Porównanie wyznaczników zestaw danych C")
plot.draw()

print_compared_points_table(compared_points_c, "Liczba punktów inaczej sklasyfikowanych przez wyznaczniki:")

#### Zestaw danych D

In [ ]:
scenes, compared_points_d = print_compare_points(classified_points_d)
plot = Plot(scenes, title="Porównanie wyznaczników zestaw danych D")
plot.draw()

print_compared_points_table(compared_points_d, "Liczba punktów inaczej sklasyfikowanych przez wyznaczniki:")

## Zbiory punktów typu float32
Domyślnie numpy używa typu `numpy.float64`, dlatego warto zobaczyć co się stanie, gdy zmniejszymy precyzję.


Zbiór A - $10^5$ losowych punktów o współrzędnych z przedziału $[-1000, 1000]$


In [ ]:
points_list_32_a = np.float32(points_list_a.points)
points_list_32_a = PointsCollection(points_list_32_a)

plot = Plot(points=[points_list_32_a], title="Zestaw danych A - float32")
plot.draw()

Zbiór B - $10^5$ losowych punktów o współrzędnych z przedziału $[-10^{14}, 10^{14}]$

In [ ]:
points_list_32_b = np.float32(points_list_b.points)
points_list_32_b = PointsCollection(points_list_32_b)

plot = Plot(points=[points_list_32_b], title="Zestaw danych B - float32")
plot.draw()

Zbiór C - $1000$ losowych punktów leżących na okręgu o środku $(0,0)$ i promieniu $R=100$

In [ ]:
points_list_32_c = np.float32(points_list_c.points)
points_list_32_c = PointsCollection(points_list_32_c)

plot = Plot(points=[points_list_32_c], title="Zestaw danych C - float32")
plot.draw()

Zbiór D - $1000$ losowych punktów o współrzędnych z przedziału $[-1000, 1000]$ leżących na prostej
wyznaczonej przez wektor (a, b).  
Przyjmij $a = [-1.0, 0.0]$, $b = [1.0, 0.1]$

In [ ]:
points_list_32_d = np.float32(points_list_d.points)
points_list_32_d = PointsCollection(points_list_32_d)

plot = Plot(points=[points_list_32_d], title="Zestaw danych D - float32")
plot.draw()

### Klasyfikacja punktów typu float32

#### Zestaw danych A

In [ ]:
scenes, classified_points_2a = print_classified_points(points_list_32_a)
plot = Plot(scenes, title=f"Klasyfikacja punktów zestaw danych A - float32")
plot.draw()

print_classified_points_table(classified_points_2a, "Tabela klasyfikacji punktów względem tolerancji zera:")

#### Zestaw danych B

In [ ]:
scenes, classified_points_2b = print_classified_points(points_list_32_b)
plot = Plot(scenes, title=f"Klasyfikacja punktów zestaw danych B - float32")
plot.draw()

print_classified_points_table(classified_points_2b, "Tabela klasyfikacji punktów względem tolerancji zera:")

#### Zestaw danych C

In [ ]:
scenes, classified_points_2c = print_classified_points(points_list_32_c)
plot = Plot(scenes, title=f"Klasyfikacja punktów zestaw danych C - float32")
plot.draw()

print_classified_points_table(classified_points_2c, "Tabela klasyfikacji punktów względem tolerancji zera:")

#### Zestaw danych D

In [ ]:
scenes, classified_points_2d = print_classified_points(points_list_32_d)
plot = Plot(scenes, title=f"Klasyfikacja punktów zestaw danych D - float32")
plot.draw()

print_classified_points_table(classified_points_2d, "Tabela klasyfikacji punktów względem tolerancji zera:")

### Porównanie klasyfikacji punktów typu float32

#### Zestaw danych A

In [ ]:
scenes, compared_points_2a = print_compare_points(classified_points_2a)
plot = Plot(scenes, title="Porównanie wyznaczników zestaw danych A - float32")
plot.draw()

print_compared_points_table(compared_points_2a, "Liczba punktów inaczej sklasyfikowanych przez wyznaczniki:")

#### Zestaw danych B

In [ ]:
scenes, compared_points_2b = print_compare_points(classified_points_2b)
plot = Plot(scenes, title="Porównanie wyznaczników zestaw danych B - float32")
plot.draw()

print_compared_points_table(compared_points_2b, "Liczba punktów inaczej sklasyfikowanych przez wyznaczniki:")

#### Zestaw danych C

In [ ]:
scenes, compared_points_2c = print_compare_points(classified_points_2c)
plot = Plot(scenes, title="Porównanie wyznaczników zestaw danych C - float32")
plot.draw()

print_compared_points_table(compared_points_2c, "Liczba punktów inaczej sklasyfikowanych przez wyznaczniki:")

#### Zestaw danych D

In [ ]:
scenes, compared_points_2d = print_compare_points(classified_points_2d)
plot = Plot(scenes, title="Porównanie wyznaczników zestaw danych D - float32")
plot.draw()

print_compared_points_table(compared_points_2d, "Liczba punktów inaczej sklasyfikowanych przez wyznaczniki:")